In [1]:
import json

with open('./data/drivers.json', 'r') as f:
    data = json.load(f)

wedding_count = 0

for driver in data:
    if 'capabilities' in driver and 'wedding' in driver['capabilities']:
        wedding_count += 1

print(f"Number of drivers tagged 'wedding': {wedding_count}")

Number of drivers tagged 'wedding': 14


In [2]:
import json

with open('./data/orders.json', 'r') as f:
    data = json.load(f)

wedding_count = 0

for order in data:
    if 'tags' in order and 'wedding' in order['tags']:
        wedding_count += 1

print(f"Number of orders tagged 'wedding': {wedding_count}")

Number of orders tagged 'wedding': 36


In [3]:
import json
from collections import defaultdict
from datetime import datetime

# Load orders
with open('./data/orders.json', 'r') as f:
    orders = json.load(f)

# Count wedding orders per date
wedding_per_date = defaultdict(int)

for order in orders:
    if 'tags' in order and 'wedding' in order['tags']:
        date = order['pickup_time'][:10]  # Extract 'YYYY-MM-DD'
        wedding_per_date[date] += 1

# Print distribution and check if within capacity
driver_capacity = 14  # Number of available drivers for wedding orders

print(f"{'Date':<12} {'Wedding Orders':<15} {'Fits Capacity?'}")
print('-' * 40)
for date in sorted(wedding_per_date):
    count = wedding_per_date[date]
    fits = "YES" if count <= driver_capacity else "NO"
    print(f"{date:<12} {count:<15} {fits}")

Date         Wedding Orders  Fits Capacity?
----------------------------------------
2024-11-02   34              NO
2024-11-03   2               YES


In [1]:
import pandas as pd
import json
from datetime import datetime

# Load data
with open('./data/orders.json') as f:
    orders = json.load(f)
with open('./data/drivers.json') as f:
    drivers = json.load(f)

# Feature engineering for orders
for o in orders:
    o['is_wedding'] = 'wedding' in o.get('tags', [])
    o['is_vip'] = 'vip' in o.get('tags', [])
    o['pickup_dt'] = datetime.fromisoformat(o['pickup_time'])
    o['start_dt'] = datetime.fromisoformat(o['setup_time'])
    o['end_dt'] = datetime.fromisoformat(o['teardown_time'])
    o['duration_hours'] = (o['end_dt'] - o['start_dt']).total_seconds() / 3600

# df_orders = pd.DataFrame([o for o in orders if o['is_wedding']])

df_orders = pd.DataFrame(orders)

# Priority score: VIP first, then pax, then shorter duration
df_orders['priority_score'] = (
    df_orders['is_vip'].astype(int) * 1000 +
    df_orders['pax_count'] +
    (10 - df_orders['duration_hours'])  # shorter duration preferred
)
df_orders = df_orders.sort_values(['priority_score'], ascending=False)

In [2]:
drivers_capable = []
for d in drivers:
    drivers_capable.append({
        **d,
        'assigned_orders': [],        # To keep assigned orders
        'assigned_on_date': {}        # date: list of (start_dt, end_dt)
    })


### Greedy

In [3]:
# --- Greedy Assignment ---
assignment = []

def is_time_conflict(new_start, new_end, assigned_slots):
    """Return True if new order overlaps any assigned slot."""
    for s, e in assigned_slots:
        if not (new_end <= s or new_start >= e):
            return True
    return False

for idx, order in df_orders.iterrows():
    order_assigned = False
    order_date = order['start_dt'].date()
    # Rank drivers: region match first, then fewest assigned on date, then highest capacity
    ranked_drivers = sorted(
        drivers_capable,
        key=lambda d: (
            d['preferred_region'] != order['region'],  # region match = 0, else 1
            len(d['assigned_on_date'].get(order_date, [])),  # fewest assigned today
            -d['max_orders_per_day']  # higher capacity preferred
        )
    )
    for driver in ranked_drivers:
        # --- Capability check for wedding orders ---
        if order['is_wedding'] and ('capabilities' not in driver or 'wedding' not in driver['capabilities']):
            continue
        assigned_today = driver['assigned_on_date'].get(order_date, [])
        if len(assigned_today) >= driver['max_orders_per_day']:
            continue  # reached max capacity
        if is_time_conflict(order['start_dt'], order['end_dt'], assigned_today):
            continue  # overlapping
        # Assign
        driver['assigned_orders'].append(order['order_id'])
        driver['assigned_on_date'].setdefault(order_date, []).append((order['start_dt'], order['end_dt']))
        assignment.append({
            'order_id': order['order_id'],
            'driver_id': driver['driver_id'],
            'order_priority': order['priority_score'],
            'pickup_dt': order['pickup_dt'],
            'end_dt': order['end_dt'],
            'is_wedding': order['is_wedding'],
            'is_vip': order['is_vip'],
            'pax_count': order['pax_count'],
            'region': order['region']
        })
        order_assigned = True
        break  # move to next order

    if not order_assigned:
        assignment.append({
            'order_id': order['order_id'],
            'driver_id': None,  # Not assigned
            'order_priority': order['priority_score'],
            'pickup_dt': order['pickup_dt'],
            'end_dt': order['end_dt'],
            'is_wedding': order['is_wedding'],
            'is_vip': order['is_vip'],
            'pax_count': order['pax_count'],
            'region': order['region']
        })

# --- Results as DataFrame ---
df_assign = pd.DataFrame(assignment)

assigned_df = df_assign[df_assign['driver_id'].notnull()]
unassigned_df = df_assign[df_assign['driver_id'].isnull()]

print("Assigned orders summary:")
display(assigned_df[['order_id', 'driver_id', 'order_priority', 'pickup_dt', 'end_dt', 'is_wedding', 'is_vip', 'pax_count', 'region']])

print(f"\nTotal assigned: {len(assigned_df)} / {len(df_assign)}")
print("Unassigned orders:")
display(unassigned_df[['order_id', 'order_priority', 'pickup_dt', 'end_dt', 'is_wedding', 'is_vip', 'pax_count', 'region']])

Assigned orders summary:


,order_id,driver_id,order_priority,pickup_dt,end_dt,is_wedding,is_vip,pax_count,region
0,Q3247,DRV-022,1207.0,2024-11-02 20:30:00,2024-11-03 00:30:00,True,True,200,north
1,Q3958,DRV-013,1207.0,2024-11-02 18:45:00,2024-11-02 22:45:00,True,True,200,north
2,Q2717,DRV-008,1207.0,2024-11-02 19:45:00,2024-11-02 23:45:00,True,True,200,east
3,Q3819,DRV-052,1207.0,2024-11-02 20:45:00,2024-11-03 00:45:00,True,True,200,east
4,P8887,DRV-041,1207.0,2024-11-02 20:15:00,2024-11-03 00:15:00,True,True,200,north
5,P6984,DRV-064,1207.0,2024-11-02 18:45:00,2024-11-02 22:45:00,True,True,200,east
6,Q4627,DRV-055,1207.0,2024-11-02 19:00:00,2024-11-02 23:00:00,True,True,200,central
7,N8029,DRV-026,1207.0,2024-11-02 18:30:00,2024-11-02 22:30:00,True,True,200,east
8,P7916,DRV-016,1187.0,2024-11-02 20:15:00,2024-11-03 00:15:00,True,True,180,east
9,P4347,DRV-038,1187.0,2024-11-02 19:30:00,2024-11-02 23:30:00,True,True,180,north



Total assigned: 40 / 60
Unassigned orders:


,order_id,order_priority,pickup_dt,end_dt,is_wedding,is_vip,pax_count,region
15,Q1619,1157.0,2024-11-02 19:45:00,2024-11-02 23:45:00,True,True,150,north
16,P4436,1157.0,2024-11-02 19:15:00,2024-11-02 23:15:00,True,True,150,east
17,O9950,1157.0,2024-11-02 18:30:00,2024-11-02 22:30:00,True,True,150,central
18,Q5082,1157.0,2024-11-02 19:00:00,2024-11-02 23:00:00,True,True,150,north
20,Q0753,127.0,2024-11-02 19:45:00,2024-11-02 23:45:00,True,False,120,east
21,P5232,127.0,2024-11-02 19:15:00,2024-11-02 23:15:00,True,False,120,north
22,Q3548,127.0,2024-11-02 18:45:00,2024-11-02 22:45:00,True,False,120,east
23,O3425,127.0,2024-11-02 19:15:00,2024-11-02 23:15:00,True,False,120,north
24,P6985,107.0,2024-11-02 20:45:00,2024-11-03 00:45:00,True,False,100,east
25,Q1717,107.0,2024-11-02 19:00:00,2024-11-02 23:00:00,True,False,100,north


### PuLP

In [11]:
import pulp
import pandas as pd
from itertools import product

# Assume df_orders and drivers_capable are already defined as before

# Step 1: Build a list of possible driver-order assignments
assignments = []
order_idx = df_orders.index
for i in order_idx:
    o = df_orders.loc[i]
    for d, driver in enumerate(drivers_capable):
        # If this is a wedding order, the driver must have wedding capability
        if o['is_wedding'] and ('capabilities' not in driver or 'wedding' not in driver['capabilities']):
            continue
        # Otherwise, any driver can be assigned
        assignments.append((i, d))

# Region match bonus
region_bonus = 10

# Step 2: Define the problem
prob = pulp.LpProblem("OrderAssignment", pulp.LpMaximize)

# Decision variables: x[i, d] = 1 if order i is assigned to driver d
x = pulp.LpVariable.dicts("assign", assignments, 0, 1, pulp.LpBinary)

# Objective: Maximize total priority score + region match bonus
prob += pulp.lpSum([
    x[(i, d)] * (
        df_orders.loc[i]['priority_score']
        + (region_bonus if drivers_capable[d]['preferred_region'] == df_orders.loc[i]['region'] else 0)
    )
    for (i, d) in assignments
])

# Constraint: Each order assigned at most once
for i in order_idx:
    prob += pulp.lpSum([x[(i, d)] for d in range(len(drivers_capable)) if (i, d) in x]) <= 1

# Constraint: Each driver can do at most max_orders_per_day per day
for d, driver in enumerate(drivers_capable):
    dates = set(df_orders['pickup_dt'])
    for date in dates:
        orders_on_date = df_orders[df_orders['pickup_dt'] == date].index
        prob += pulp.lpSum([x[(i, d)] for i in orders_on_date if (i, d) in x]) <= driver['max_orders_per_day']

# Constraint: No driver can take overlapping orders (precompute conflict pairs)
def overlap(o1, o2):
    return not (o1['end_dt'] <= o2['start_dt'] or o1['start_dt'] >= o2['end_dt'])

for d, driver in enumerate(drivers_capable):
    for i1, i2 in product(df_orders.index, repeat=2):
        if i1 < i2:
            o1, o2 = df_orders.loc[i1], df_orders.loc[i2]
            # If this is a wedding order, the driver must have wedding capability
            if (o1['is_wedding'] and ('capabilities' not in driver or 'wedding' not in driver['capabilities'])):
                continue
            if (o2['is_wedding'] and ('capabilities' not in driver or 'wedding' not in driver['capabilities'])):
                continue
            # Now check overlap
            if overlap(o1, o2):
                if (i1, d) in x and (i2, d) in x:
                    prob += x[(i1, d)] + x[(i2, d)] <= 1

# Solve!
prob.solve()

# Extract assignment results
assigned = []
for (i, d), var in x.items():
    if var.varValue > 0.5:
        assigned.append((df_orders.loc[i]['order_id'], drivers_capable[d]['driver_id']))

print(f"Assigned {len(assigned)} orders out of {len(df_orders)} (ILP)")

# Show unassigned orders
assigned_ids = set(a[0] for a in assigned)
unassigned_orders = df_orders[~df_orders['order_id'].isin(assigned_ids)]
print(unassigned_orders[['order_id', 'priority_score', 'is_vip', 'pax_count', 'region']])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yeo_menghan/Documents/personal-projects/grain-project/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/_l/9yt9fjns3yl8khq3hlylkj7w0000gn/T/a51c59890bfe42e8b81f7aba7835bfc6-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/_l/9yt9fjns3yl8khq3hlylkj7w0000gn/T/a51c59890bfe42e8b81f7aba7835bfc6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 26357 COLUMNS
At line 85942 RHS
At line 112295 BOUNDS
At line 114480 ENDATA
Problem MODEL has 26352 rows, 2184 columns and 53032 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 26099 - 0.21 seconds
Cgl0008I 60 inequality constraints converted to equality constraints
Cgl0003I 0 fixed, 0 tightened bounds, 22733 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds,

KeyboardInterrupt: 

In [7]:
import pandas as pd
from collections import defaultdict

# 'assigned' is a list of (order_id, driver_id)
df_assigned = pd.DataFrame(assigned, columns=['order_id', 'driver_id'])

# Merge with order details (make sure to use the correct field names)
df_assigned_details = df_assigned.merge(
    df_orders,
    on='order_id',
    how='left'
)

# List of fields you want
fields = [
    'order_id', 'priority_score', 'pickup_dt', 'end_dt',
    'is_wedding', 'is_vip', 'pax_count', 'region'
]

# Print assignments grouped by driver
for driver_id, group in df_assigned_details.groupby('driver_id'):
    print(f"\nDriver {driver_id} assigned orders:")
    print(group[fields].sort_values('pickup_dt').to_string(index=False))


Driver DRV-001 assigned orders:
order_id  priority_score           pickup_dt              end_dt  is_wedding  is_vip  pax_count region
   Q3372            32.0 2024-11-02 19:00:00 2024-11-02 23:00:00       False   False         25  north

Driver DRV-002 assigned orders:
order_id  priority_score           pickup_dt              end_dt  is_wedding  is_vip  pax_count region
   Q1662            67.0 2024-11-02 20:15:00 2024-11-03 00:15:00       False   False         60   west

Driver DRV-008 assigned orders:
order_id  priority_score           pickup_dt              end_dt  is_wedding  is_vip  pax_count region
   N8029          1207.0 2024-11-02 18:30:00 2024-11-02 22:30:00        True    True        200   east
   P2994           127.0 2024-11-03 02:00:00 2024-11-03 06:00:00        True   False        120   east

Driver DRV-011 assigned orders:
order_id  priority_score           pickup_dt              end_dt  is_wedding  is_vip  pax_count region
   P9766            47.0 2024-11-03 01:45:00

In [8]:
all_driver_ids = {d['driver_id'] for d in drivers_capable}
allocated_driver_ids = set(df_assigned['driver_id'])
unallocated_driver_ids = all_driver_ids - allocated_driver_ids

print("Drivers with no assigned orders:")
for driver_id in unallocated_driver_ids:
    print(driver_id)

Drivers with no assigned orders:
DRV-028
DRV-010
DRV-045
DRV-063
DRV-021
DRV-054
DRV-004
DRV-034
DRV-070
DRV-059
DRV-044
DRV-024
DRV-019
DRV-003
DRV-053
DRV-047
DRV-048
DRV-057
DRV-067
DRV-009
DRV-023
DRV-035
DRV-016
DRV-062
DRV-006
DRV-032
DRV-005
DRV-058
DRV-066
DRV-030
DRV-060
DRV-007
DRV-037
DRV-061
DRV-051
DRV-046
DRV-068
DRV-069
DRV-029
DRV-031
DRV-039
DRV-056
DRV-033


In [9]:
assigned_ids = set(order_id for order_id, driver_id in assigned)
unassigned_orders = df_orders[~df_orders['order_id'].isin(assigned_ids)]

columns = [
    'order_id', 'priority_score', 'pickup_dt', 'end_dt',
    'is_wedding', 'is_vip', 'pax_count', 'region'
]
display(unassigned_orders[columns])

,order_id,priority_score,pickup_dt,end_dt,is_wedding,is_vip,pax_count,region
14,Q3247,1207.0,2024-11-02 20:30:00,2024-11-03 00:30:00,True,True,200,north
9,Q3958,1207.0,2024-11-02 18:45:00,2024-11-02 22:45:00,True,True,200,north
26,P7916,1187.0,2024-11-02 20:15:00,2024-11-03 00:15:00,True,True,180,east
30,P4347,1187.0,2024-11-02 19:30:00,2024-11-02 23:30:00,True,True,180,north
10,N8347,1157.0,2024-11-02 19:00:00,2024-11-02 23:00:00,True,True,150,central
38,K9492,1157.0,2024-11-02 18:00:00,2024-11-02 22:00:00,True,True,150,central
11,Q1619,1157.0,2024-11-02 19:45:00,2024-11-02 23:45:00,True,True,150,north
18,P4436,1157.0,2024-11-02 19:15:00,2024-11-02 23:15:00,True,True,150,east
50,Q5082,1157.0,2024-11-02 19:00:00,2024-11-02 23:00:00,True,True,150,north
13,Q0753,127.0,2024-11-02 19:45:00,2024-11-02 23:45:00,True,False,120,east


In [10]:
for idx, row in unassigned_orders.iterrows():
    possible_drivers = []
    for d, driver in enumerate(drivers_capable):
        # Region match
        if driver['preferred_region'] != row['region']:
            continue
        # Wedding capability
        if row['is_wedding'] and ('capabilities' not in driver or 'wedding' not in driver['capabilities']):
            continue
        # Max orders per day (you may want to count by date)
        # Overlap: in ILP, this is handled by constraints
        possible_drivers.append(driver['driver_id'])
    print(f"Order {row['order_id']} possible drivers: {possible_drivers}")

Order Q3247 possible drivers: ['DRV-022']
Order Q3958 possible drivers: ['DRV-022']
Order P7916 possible drivers: ['DRV-008', 'DRV-026', 'DRV-052', 'DRV-064']
Order P4347 possible drivers: ['DRV-022']
Order N8347 possible drivers: ['DRV-043', 'DRV-055']
Order K9492 possible drivers: ['DRV-043', 'DRV-055']
Order Q1619 possible drivers: ['DRV-022']
Order P4436 possible drivers: ['DRV-008', 'DRV-026', 'DRV-052', 'DRV-064']
Order Q5082 possible drivers: ['DRV-022']
Order Q0753 possible drivers: ['DRV-008', 'DRV-026', 'DRV-052', 'DRV-064']
Order P5232 possible drivers: ['DRV-022']
Order Q3548 possible drivers: ['DRV-008', 'DRV-026', 'DRV-052', 'DRV-064']
Order O3425 possible drivers: ['DRV-022']
Order P6985 possible drivers: ['DRV-008', 'DRV-026', 'DRV-052', 'DRV-064']
Order Q1717 possible drivers: ['DRV-022']
Order P9688 possible drivers: ['DRV-013', 'DRV-038', 'DRV-041']
Order P9890 possible drivers: ['DRV-022']
Order Q4974 possible drivers: ['DRV-008', 'DRV-026', 'DRV-052', 'DRV-064']
Or

### PuLP with fairness penalty

In [ ]:
import pulp
import pandas as pd
from itertools import product

# ... assume df_orders and drivers_capable are already defined ...

# Step 1: Build a list of possible driver-order assignments
assignments = []
order_idx = df_orders.index
for i in order_idx:
    o = df_orders.loc[i]
    for d, driver in enumerate(drivers_capable):
        if driver['preferred_region'] == o['region'] and 'wedding' in driver['capabilities']:
            assignments.append((i, d))

# Step 2: Define the problem
prob = pulp.LpProblem("OrderAssignment", pulp.LpMaximize)

# Decision variables: x[i, d] = 1 if order i is assigned to driver d
x = pulp.LpVariable.dicts("assign", assignments, 0, 1, pulp.LpBinary)

# Fairness penalty parameter (tune this value as needed)
FAIRNESS_PENALTY = 0.01

# --- Add job count variables for fairness penalty ---
job_count_vars = {}
for d in range(len(drivers_capable)):
    job_count_vars[d] = pulp.LpVariable(f"job_count_{d}", lowBound=0, cat='Integer')
    # job_count_vars[d] = sum of assignments to driver d
    prob += job_count_vars[d] == pulp.lpSum([x[(i, d)] for (i, dd) in assignments if dd == d])

# Objective: Maximize total priority score, penalize unbalanced driver loads (sum of squares)
prob += (
    pulp.lpSum([
        x[(i, d)] * df_orders.loc[i]['priority_score']
        for (i, d) in assignments
    ])
    - FAIRNESS_PENALTY * pulp.lpSum([job_count_vars[d] ** 2 for d in range(len(drivers_capable))])
)

# Constraint: Each order assigned at most once
for i in order_idx:
    prob += pulp.lpSum([x[(i, d)] for d in range(len(drivers_capable)) if (i, d) in x]) <= 1

# Constraint: Each driver can do at most max_orders_per_day per day
for d, driver in enumerate(drivers_capable):
    dates = set(df_orders['pickup_dt'])
    for date in dates:
        orders_on_date = df_orders[df_orders['pickup_dt'] == date].index
        prob += pulp.lpSum([x[(i, d)] for i in orders_on_date if (i, d) in x]) <= driver['max_orders_per_day']

# Constraint: No driver can take overlapping orders (precompute conflict pairs)
def overlap(o1, o2):
    return not (o1['end_dt'] <= o2['start_dt'] or o1['start_dt'] >= o2['end_dt'])

for d, driver in enumerate(drivers_capable):
    for date in set(df_orders['pickup_dt']):
        orders_on_date = df_orders[df_orders['pickup_dt'] == date]
        for i1, i2 in product(orders_on_date.index, repeat=2):
            if i1 < i2:
                o1, o2 = df_orders.loc[i1], df_orders.loc[i2]
                if overlap(o1, o2):
                    # Both not assigned to the same driver
                    if (i1, d) in x and (i2, d) in x:
                        prob += x[(i1, d)] + x[(i2, d)] <= 1

# Solve!
prob.solve()

# Extract assignment results
assigned = []
for (i, d), var in x.items():
    if var.varValue > 0.5:
        assigned.append((df_orders.loc[i]['order_id'], drivers_capable[d]['driver_id']))

print(f"Assigned {len(assigned)} orders out of {len(df_orders)} (ILP)")

# Show unassigned orders
assigned_ids = set(a[0] for a in assigned)
unassigned_orders = df_orders[~df_orders['order_id'].isin(assigned_ids)]
print(unassigned_orders[['order_id', 'priority_score', 'is_vip', 'pax_count', 'region']])

# Show number of orders per driver
import pandas as pd
df_assigned = pd.DataFrame(assigned, columns=['order_id', 'driver_id'])
order_counts = df_assigned['driver_id'].value_counts()
print("\nNumber of orders assigned per driver:")
print(order_counts)